In [1]:
import pandas as pd
import numpy as np

In [2]:
#pyspark 
from pyspark import SparkContext
sc = SparkContext()

In [3]:
from pyspark.mllib.regression import LabeledPoint #라벨을 표시하기 위해 사용
from pyspark.mllib.feature import HashingTF #단어를 숫자로 변환하고 빈도수를 매핑하기 위해 사용
from pyspark.mllib.classification import LogisticRegressionWithSGD #SGD 즉 확률론적경사하강법을 적용한 로지스틱 회귀분석을 위해 사용

#파일읽기

spam = sc.textFile("C:/spark/data/mllib/spam.txt")

normal = sc.textFile("C:/spark/data/mllib/normal.txt")


In [4]:
#파일읽기
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/reejungkim/Natural-Language-Processing/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv',encoding='latin1')
data.head() #data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
spam = data.loc[data.v1==1]
normal = data.loc[data.v1==0]

In [7]:
spam.head()

,v1,v2
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
5,1,FreeMsg Hey there darling it's been 3 week's n...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...
11,1,"SIX chances to win CASH! From 100 to 20,000 po..."


In [8]:
normal.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
6,0,Even my brother is not like to speak with me. ...


## read file using SparkContext

In [9]:
inputFile = sc.textFile("/Users/reejungkim/Documents/Git/NLP/spam.csv")

In [10]:
# retrieve and remove header line
header = inputFile.first()
inputFile = inputFile.filter(lambda line: line != header)

In [11]:
def loadRecord(line):
    inputLine = StringIO(line)
    reader = csv.DictReader(inputLine, fieldnames=header.split(","))
    return next(reader)

In [12]:
inputCSV = inputFile.map(loadRecord)
inputCSV.first()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/reejungkim/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/Users/reejungkim/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/reejungkim/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/reejungkim/opt/anaconda3/envs/mye

In [ ]:
#HashingTF 기능 사용하기 위해 축약어로 정의
tf = HashingTF()

In [ ]:
#문장을 빈칸을 기준으로 자른 후, 단어별로 몇 번 나왔는지를 체크하는 함수

spamFeatures = spam.map(lambda email : tf.transform(email.split(" ")))
normalFeatures = normal.map(lambda email : tf.transform(email.split(" ")))

In [13]:
#라벨 표시

spamExamples = spamFeatures.map(lambda features : LabeledPoint(1, features))
normalExamples = normalFeatures.map(lambda features : LabeledPoint(0, features))

NameError: name 'spamFeatures' is not defined

In [ ]:
#두 데이터를 하나로 합침

trainingData = spamExamples.union(normalExamples)
trainingData.cache() #메모리에 올려놓기

In [ ]:
#학습데이터로 모델만들기

model = LogisticRegressionWithSGD.train(trainingData)

In [ ]:
#테스트 데이터 만들기

spamTest = tf.transform("Dear sir, I am a Prince in a far kingdom you have not heard of.".split(" "))
normalTest = tf.transform("Patrick, How are you?  Love, Dad".split(" "))

In [ ]:
#결과 확인

print("Prediction for positive test example: %f" % model.predict(spamTest))
print("Prediction for negative test example: %f" % model.predict(normalTest))




In [31]:
sc.stop()

import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()
